In [107]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from datetime import datetime
import itertools

In [108]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
503,tt0240772,85000000,450717150,Ocean's Eleven,George Clooney|Brad Pitt|Matt Damon|Andy GarcÃ...,Steven Soderbergh,Are you in or out?,"Less than 24 hours into his parole, charismati...",116,Thriller|Crime,Village Roadshow Pictures|Section Eight|NPV En...,12/7/2001,7.0,2001
719,tt0494652,35000000,43650785,Welcome Home Roscoe Jenkins,Martin Lawrence|James Earl Jones|Margaret Aver...,Malcolm D. Lee,Roscoe Jenkins aims for the heartstrings and f...,"Martin Lawrence leads an all-star cast, includ...",114,Comedy|Drama,Universal Pictures|Spyglass Entertainment,2/8/2008,6.1,2008
492,tt1391137,20000000,60072596,Why Did I Get Married Too?,Tyler Perry|Janet Jackson|Jill Scott|Sharon Le...,Tyler Perry,Marriage is an institution they're committed to.,Four couples reunite for their annual vacation...,121,Comedy|Romance,Lions Gate Films|The Tyler Perry Company,4/2/2010,6.0,2010
509,tt0252866,30000000,287553595,American Pie 2,Jason Biggs|Thomas Ian Nicholas|Chris Klein|Se...,J.B. Rogers,This Summer It's All About Sticking Together.,The whole gang are back and as close as ever. ...,108,Comedy|Romance,Universal Pictures|Liveplanet|Zide-Perry Produ...,8/10/2001,5.9,2001
1554,tt0417433,26000000,17496992,Annapolis,James Franco|Donnie Wahlberg|Jim Parrack|Tyres...,Justin Lin,"50,000 Apply. 1,200 Are Accepted. Only The Bes...","Jake Huard, from a shipbuilders family, promis...",108,Drama,Touchstone Pictures,1/27/2006,5.8,2006


In [109]:
display(data.describe())
display(data.info())

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
imdb_id                 1889 non-null object
budget                  1889 non-null int64
revenue                 1889 non-null int64
original_title          1889 non-null object
cast                    1889 non-null object
director                1889 non-null object
tagline                 1889 non-null object
overview                1889 non-null object
runtime                 1889 non-null int64
genres                  1889 non-null object
production_companies    1889 non-null object
release_date            1889 non-null object
vote_average            1889 non-null float64
release_year            1889 non-null int64
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


None

# Предобработка

In [110]:
answers = {}  # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

# the time given in the dataset is in string format.
# So we need to change this in datetime format
# ...

In [111]:
# Дополнительный столбец: прибыль от фильма
data['profit'] = data['revenue'] - data['budget']

In [112]:
# Выборки за 2008 и 2012 годы
data_2008 = data[data.release_year == 2008]
data_2012 = data[data.release_year == 2012]

In [113]:
# Выборки за период с 2012 по 2014 годы
data_2012_2014 = data[
    (data.release_year >= 2012)
    & (data.release_year <= 2014)
]

In [114]:
# Счетчик жанров
genres_counter = Counter()
for genres in data.genres:
    for genre in genres.split('|'):
        genres_counter[genre] += 1

In [115]:
# Счетчик жанров, с которых была получена прибыль
genres_profit_counter = Counter()
for _, row in data.iterrows():
    if row.profit > 0:
        for genre in row.genres.split('|'):
            genres_profit_counter[genre] += 1

In [116]:
# Суммарные кассовые сборы фильмов по режисерам
director_revenue_total = Counter()
for _, row in data.iterrows():
    for director in row.director.split('|'):
        director_revenue_total[director] += row.revenue

In [117]:
# Счетчи режисеров, снявших Action
director_action_counter = Counter()
for _, row in data.iterrows():
    if "Action" in row.genres:
        for director in row.director.split('|'):
            director_action_counter[director] += 1

In [118]:
# Суммарные кассовые сборы фильмов по актерам за 2012
actor_revenue_total_2012 = Counter()
for _, row in data_2012.iterrows():
    for actor in row.cast.split('|'):
        actor_revenue_total_2012[actor] += row.revenue

In [119]:
# Выборка высокобюджетных фильмов
data_high_budget = data[data.budget > data.budget.mean()]

In [120]:
# Счетчи высокобюджетных фильмов по актерам
actor_high_budget = Counter()
for cast in data_high_budget.cast:
    for actor in cast.split('|'):
        actor_high_budget[actor] += 1

In [121]:
# Счетчики жанров фильмов по актерам
actor_genre_counter = {}
for _, row in data.iterrows():
    for actor in row.cast.split('|'):
        actor_genre_counter[actor] = actor_genre_counter.setdefault(
            actor, Counter())
        for genre in row.genres.split('|'):
            actor_genre_counter[actor][genre] += 1

In [122]:
# Выборка фильмов с участием Paramount Pictures
data_pp = data[data.production_companies.str.contains('Paramount Pictures')]

In [123]:
# Выборка фильмов с участием Warner Bros
data_wb = data[data.production_companies.str.contains('Warner Bros')]

In [124]:
# Дополнительные столбцы: полная дата релиза, масяц релиза, сезон года
data['release_date_dt'] = data.release_date.apply(
    lambda d: datetime.strptime(d, "%m/%d/%Y"))
data['release_month'] = data.release_date_dt.apply(
    lambda d: datetime.strftime(d, "%B"))

In [125]:
# Дополнительный столбец: сезон года
def get_season(date):
    month_num = int(datetime.strftime(date, "%-m"))
    if month_num in (3, 4, 5):
        return 'Spring'
    elif month_num in (6, 7, 8):
        return 'Summer'
    elif month_num in (9, 10, 11):
        return 'Fall'
    else:
        return 'Winter'


data['release_date_season'] = data.release_date_dt.apply(get_season)

In [126]:
# Счетчки фильмов по сезонам и режисерам
season_director_counter = {}
for _, row in data.iterrows():
    season = row.release_date_season
    season_director_counter[season] = season_director_counter.setdefault(
        season, Counter())
    for actor in row.director.split('|'):
        season_director_counter[season][actor] += 1

In [127]:
# Дополнительные столбецы: длина названия и число слов в описании
data['original_title_length'] = data.original_title.apply(
    lambda text: len(text)
)
data['overview_words'] = data.overview.apply(
    lambda text: len(text.split())
)

In [128]:
# Длины названий и средние длины названий по студиям
prodcomp_title_len = {}
prodcomp_title_len_mean = {}

# Число слов в описании и среднее число слов в описании по студиям
prodcomp_overview_words = {}
prodcomp_overview_words_mean = {}

# Заполняем словари длин названий (prodcomp_title_len) и числа слов описания (prodcomp_overview_words)
for _, row in data.iterrows():
    for prod_comp in row.production_companies.split('|'):
        prodcomp_title_len[prod_comp] = prodcomp_title_len.setdefault(
            prod_comp, [])
        prodcomp_title_len[prod_comp].append(row.original_title_length)

        prodcomp_overview_words[prod_comp] = prodcomp_overview_words.setdefault(
            prod_comp, [])
        prodcomp_overview_words[prod_comp].append(row.overview_words)

# Cредние длины названий по студиям (prodcomp_title_len_mean)
for prod_comp, lengthes in prodcomp_title_len.items():
    prodcomp_title_len_mean[prod_comp] = sum(lengthes)/len(lengthes)

# Среднее число слов в описании по студиям (prodcomp_overview_words_mean)
for prod_comp, words in prodcomp_overview_words.items():
    prodcomp_overview_words_mean[prod_comp] = sum(words)/len(words)

In [129]:
# Счетчик пар акторов, снимающихся вместе
actor_pairs = Counter()
for cast in data.cast:
    actors = cast.split('|')
    pairs = [' & '.join(c) for c in itertools.combinations(actors, 2)]
    for pair in pairs:
        actor_pairs[pair] += 1

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [130]:
budget_max = data[data.budget == data.budget.max()]
budget_max[['imdb_id', 'original_title']]

,imdb_id,original_title
723,tt1298650,Pirates of the Caribbean: On Stranger Tides


In [131]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример:
# answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '5. {} ({})'.format(
    budget_max.iloc[0]['original_title'],
    budget_max.iloc[0]['imdb_id']
)
# если ответили верно, можете добавить комментарий со значком "+"

In [132]:
answers['1']

'5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'

# 2. Какой из фильмов самый длительный (в минутах)?

In [133]:
runtime_max = data[data.runtime == data.runtime.max()]
runtime_max[['imdb_id', 'original_title']]

,imdb_id,original_title
1157,tt0279111,Gods and Generals


In [134]:
# думаю логику работы с этим словарем вы уже поняли,
# по этому не буду больше его дублировать
answers['2'] = '2. {} ({})'.format(
    runtime_max.iloc[0]['original_title'],
    runtime_max.iloc[0]['imdb_id']
)

In [135]:
answers['2']

'2. Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [136]:
runtime_min = data[data.runtime == data.runtime.min()]
runtime_min[['imdb_id', 'original_title']]

,imdb_id,original_title
768,tt1449283,Winnie the Pooh


In [137]:
answers['3'] = '3. {} ({})'.format(
    runtime_min.iloc[0]['original_title'],
    runtime_min.iloc[0]['imdb_id']
)

In [138]:
answers['3']

'3. Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [139]:
answers['4'] = '2. {}'.format(
    round(data.runtime.mean())
)

In [140]:
answers['4']

'2. 110'

# 5. Каково медианное значение длительности фильмов? 

In [141]:
answers['5'] = '1. {}'.format(
    round(data.runtime.median())
)

In [142]:
answers['5']

'1. 107'

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [143]:
profit_max = data[data.profit == data.profit.max()]
profit_max[['imdb_id', 'original_title', 'profit']]

,imdb_id,original_title,profit
239,tt0499549,Avatar,2544505847


In [144]:
answers['6'] = '5. {} ({})'.format(
    profit_max.iloc[0]['original_title'],
    profit_max.iloc[0]['imdb_id']
)

In [145]:
answers['6']

'5. Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [146]:
profit_min = data[data.profit == data.profit.min()]
profit_min[['imdb_id', 'original_title', 'profit']]

,imdb_id,original_title,profit
1245,tt1210819,The Lone Ranger,-165710090


In [147]:
answers['7'] = '5. {} ({})'.format(
    profit_min.iloc[0]['original_title'],
    profit_min.iloc[0]['imdb_id']
)

In [148]:
answers['7']

'5. The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [149]:
answers['8'] = '1. {}'.format(
    len(data[data.profit > 0].index)
)

In [150]:
answers['8']

'1. 1478'

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [151]:
profit_max_2008 = data_2008[
    data_2008.profit == data_2008.profit.max()
]
profit_max_2008[['imdb_id', 'original_title',
                 'profit', 'revenue']]

,imdb_id,original_title,profit,revenue
599,tt0468569,The Dark Knight,816921825,1001921825


In [152]:
answers['9'] = '4. {} ({})'.format(
    profit_max_2008.iloc[0]['original_title'],
    profit_max_2008.iloc[0]['imdb_id']
)

In [153]:
answers['9']

'4. The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [154]:
profit_min_2012_2014 = data_2012_2014[
    data_2012_2014.profit == data_2012_2014.profit.min()
]
profit_min_2012_2014[['imdb_id', 'original_title',
                      'profit', 'revenue']]

,imdb_id,original_title,profit,revenue
1245,tt1210819,The Lone Ranger,-165710090,89289910


In [155]:
answers['10'] = '5. {} ({})'.format(
    profit_min_2012_2014.iloc[0]['original_title'],
    profit_min_2012_2014.iloc[0]['imdb_id']
)

In [156]:
answers['10']

'5. The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [157]:
genres_counter.most_common()[:1]

[('Drama', 782)]

ВАРИАНТ 2

In [158]:
answers['11'] = '3. ' \
    + genres_counter.most_common()[0][0]

In [159]:
answers['11']

'3. Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [160]:
genres_profit_counter.most_common()[:1]

[('Drama', 560)]

In [161]:
answers['12'] = '1. ' \
    + genres_profit_counter.most_common()[0][0]

In [162]:
answers['12']

'1. Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [163]:
director_revenue_total.most_common()[:1]

[('Peter Jackson', 6490593685)]

ВАРИАНТ 2

In [164]:
data.groupby('director')['revenue'].sum() \
    .sort_values(ascending=False).head(1)

director
Peter Jackson    6490593685
Name: revenue, dtype: int64

In [165]:
answers['13'] = '5. ' \
    + director_revenue_total.most_common()[0][0]

In [166]:
answers['13']

'5. Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [167]:
director_action_counter.most_common()[:1]

[('Robert Rodriguez', 9)]

In [168]:
answers['14'] = '3. ' \
    + director_action_counter.most_common()[0][0]

In [169]:
answers['14']

'3. Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [170]:
actor_revenue_total_2012.most_common()[:1]

[('Chris Hemsworth', 2027450773)]

In [171]:
answers['15'] = '3. ' \
    + actor_revenue_total_2012.most_common()[0][0]

In [172]:
answers['15']

'3. Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [173]:
actor_high_budget.most_common()[:1]

[('Matt Damon', 18)]

In [174]:
answers['16'] = '3. ' \
    + actor_high_budget.most_common()[0][0]

In [175]:
answers['16']

'3. Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [176]:
actor_genre_counter['Nicolas Cage'].most_common()[:1]

[('Action', 17)]

In [177]:
answers['17'] = '2. ' \
    + actor_genre_counter['Nicolas Cage'].most_common()[0][0]

In [178]:
answers['17']

'2. Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [179]:
pp_profit_min = data_pp[
    data_pp.profit == data_pp.profit.min()
]
pp_profit_min[['imdb_id', 'original_title', 'profit']]

,imdb_id,original_title,profit
925,tt0267626,K-19: The Widowmaker,-64831034


In [180]:
answers['18'] = '1. {} ({})'.format(
    pp_profit_min.iloc[0]['original_title'],
    pp_profit_min.iloc[0]['imdb_id']
)

In [181]:
answers['18']

'1. K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [182]:
profit_year = data.groupby('release_year')['profit'] \
    .sum().sort_values(ascending=False).head(1)
profit_year

release_year
2015    18668572378
Name: profit, dtype: int64

In [183]:
answers['19'] = '5. {}'.format(
    profit_year.index[0]
)

In [184]:
answers['19']

'5. 2015'

# 20. Какой самый прибыльный год для студии Warner Bros?

In [185]:
profit_year_wb = data_wb.groupby('release_year')['profit'] \
    .sum().sort_values(ascending=False).head(1)
profit_year_wb

release_year
2014    2295464519
Name: profit, dtype: int64

In [186]:
answers['20'] = '1. {}'.format(
    profit_year_wb.index[0]
)

In [187]:
answers['20']

'1. 2014'

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [188]:
count_release_month = data.groupby('release_month')['original_title'] \
    .count().sort_values(ascending=False)
count_release_month

release_month
September    227
December     190
October      186
August       161
March        156
April        149
June         147
November     146
July         142
May          140
February     135
January      110
Name: original_title, dtype: int64

In [189]:
answers['21'] = '4. ' \
    + count_release_month.index[0]

In [190]:
answers['21']

'4. September'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [191]:
answers['22'] = '2. {}'.format(
    count_release_month['June']
    + count_release_month['July']
    + count_release_month['August']
)

In [192]:
answers['22']

'2. 450'

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [193]:
season_director_counter['Winter'].most_common()[:1]

[('Peter Jackson', 7)]

In [194]:
answers['23'] = '5. ' \
    + season_director_counter['Winter'].most_common()[0][0]

In [195]:
answers['23']

'5. Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [196]:
longest_title_prod = sorted(
    prodcomp_title_len_mean.items(),
    key=lambda i: i[1], reverse=True)[:1]
longest_title_prod

[('Four By Two Productions', 83.0)]

In [197]:
answers['24'] = '5. ' \
    + longest_title_prod[0][0]

In [198]:
answers['24']

'5. Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [199]:
longest_over_words_prod = sorted(
    prodcomp_overview_words_mean.items(),
    key=lambda i: i[1], reverse=True)[:1]
longest_over_words_prod

[('Midnight Picture Show', 175.0)]

In [200]:
answers['25'] = '3. ' \
    + longest_over_words_prod[0][0]

In [201]:
answers['25']

'3. Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [202]:
options = {
    1: set(['Inside Out', 'The Dark Knight', '12 Years a Slave']),
    2: set(['BloodRayne', 'The Adventures of Rocky & Bullwinkle']),
    3: set(['Batman Begins', 'The Lord of the Rings: The Return of the King', 'Upside Down']),
    4: set(['300, Lucky Number Slevin', 'Kill Bill: Vol. 1']),
    5: set(['Upside Down', 'Inside Out', 'Iron Man']),
}

In [203]:
vote_average_1p = data[
    data.vote_average > np.percentile(data.vote_average, 99)
]

In [204]:
suitable_options = {}
titles_1p = set(vote_average_1p['original_title'].values)
for id, movies in options.items():
    check = titles_1p & movies
    if len(check) == len(movies):
        suitable_options[id] = movies

In [205]:
suitable_options

{1: {'12 Years a Slave', 'Inside Out', 'The Dark Knight'}}

In [206]:
answers['26'] = '1. ' \
    + ', '.join(suitable_options[1])

In [207]:
answers['26']

'1. Inside Out, The Dark Knight, 12 Years a Slave'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [208]:
actor_pairs.most_common()[:1]

[('Daniel Radcliffe & Rupert Grint', 8)]

In [209]:
answers['27'] = '5. ' \
    + actor_pairs.most_common()[0][0]

In [210]:
answers['27']

'5. Daniel Radcliffe & Rupert Grint'

# Submission

In [211]:
# в конце можно посмотреть свои ответы к каждому вопросу
for id in sorted(answers, key=lambda x: int(x)):
    print('{}: {}'.format(id, answers[id]))

1: 5. Pirates of the Caribbean: On Stranger Tides (tt1298650)
2: 2. Gods and Generals (tt0279111)
3: 3. Winnie the Pooh (tt1449283)
4: 2. 110
5: 1. 107
6: 5. Avatar (tt0499549)
7: 5. The Lone Ranger (tt1210819)
8: 1. 1478
9: 4. The Dark Knight (tt0468569)
10: 5. The Lone Ranger (tt1210819)
11: 3. Drama
12: 1. Drama
13: 5. Peter Jackson
14: 3. Robert Rodriguez
15: 3. Chris Hemsworth
16: 3. Matt Damon
17: 2. Action
18: 1. K-19: The Widowmaker (tt0267626)
19: 5. 2015
20: 1. 2014
21: 4. September
22: 2. 450
23: 5. Peter Jackson
24: 5. Four By Two Productions
25: 3. Midnight Picture Show
26: 1. Inside Out, The Dark Knight, 12 Years a Slave
27: 5. Daniel Radcliffe & Rupert Grint


In [212]:
# и убедиться что ни чего не пропустил)
len(answers)

27